In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# shared_folder_path = '/content/drive/MyDrive/Envision_Dataset'
# # https://drive.google.com/drive/folders/1Vq1Oj9-y2MmQOCc1Py0_SpwEefT7bdS3?usp=drive_link
# # List contents of the shared folder
# import os
# os.listdir(shared_folder_path)
# import shutil
# shutil.copytree(shared_folder_path, '/content/drive/MyDrive/Envision2024/dataset')  # Replace '/content/data' with your desired destination path

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
# Define the path to the audio data
data_path = '/content/drive/MyDrive/Envision2024/dataset'
def load_data(dataset_folder):
    data = []
    labels = []
    max_length = 0  # Initialize max_length to track the maximum length
    for species_folder in os.listdir(dataset_folder):
        species_path = os.path.join(dataset_folder, species_folder)
        if os.path.isdir(species_path):
            for filename in os.listdir(species_path):
                audio_path = os.path.join(species_path, filename)
                signal, sr = librosa.load(audio_path, sr=None)
                mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
                data.append(mfccs.T)  # Transpose MFCCs to match shape (time_steps, n_mfcc)
                labels.append(species_folder)
                max_length = max(max_length, mfccs.shape[1])  # Update max_length
    return data, labels, max_length

In [ ]:
data, labels, max_length = load_data(data_path)
labels[0]

'Dicrurus andamanensis'

In [ ]:
# Assuming max_length is the desired length for all sequences
for i in range(len(data)):
    if data[i].shape[0] < max_length:
        # Pad shorter sequences with zeros
        padding = np.zeros((max_length - data[i].shape[0], data[i].shape[1]))
        data[i] = np.vstack((data[i], padding))
    elif data[i].shape[0] > max_length:
        # Truncate longer sequences
        data[i] = data[i][:max_length]


In [ ]:
labels = np.array(labels)
data = np.array(data)

In [ ]:
print(data.shape)
# data = np.expand_dims(data, axis=-1)

(129, 15028, 13)


In [ ]:
data_copy = data

In [ ]:
labels[0]

'Dicrurus andamanensis'

In [ ]:
from sklearn.preprocessing import LabelEncoder


# Assuming labels is a list or array containing your original labels
# Convert labels to integer encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Get the number of unique classes
num_classes = len(label_encoder.classes_)

# Convert integer labels to categorical format using one-hot encoding
labels_categorical = to_categorical(labels_encoded, num_classes=num_classes)


In [ ]:
num_classes

5

In [ ]:
data_copy.shape

(129, 15028, 13)

# Cnn2d 2 conv layers  one flat and 1 dense layer no dropout

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_copy, labels_categorical, test_size=0.2, random_state=42)  # Split data

X_train = X_train.reshape(-1, 15028, 13, 1)
X_test = X_test.reshape(-1, 15028, 13, 1)

In [ ]:
X_train.shape

(103, 15028, 13, 1)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(15028, 13, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)


loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/10
4/4 [==============================] - 9s 725ms/step - loss: 370.3412 - accuracy: 0.2136 - val_loss: 146.6017 - val_accuracy: 0.2308
Epoch 2/10
4/4 [==============================] - 0s 93ms/step - loss: 74.2439 - accuracy: 0.2330 - val_loss: 13.5259 - val_accuracy: 0.3846
Epoch 3/10
4/4 [==============================] - 0s 93ms/step - loss: 8.6498 - accuracy: 0.4757 - val_loss: 8.5996 - val_accuracy: 0.3462
Epoch 4/10
4/4 [==============================] - 0s 92ms/step - loss: 3.5708 - accuracy: 0.5922 - val_loss: 4.5495 - val_accuracy: 0.3462
Epoch 5/10
4/4 [==============================] - 0s 91ms/step - loss: 1.6689 - accuracy: 0.7476 - val_loss: 2.4082 - val_accuracy: 0.3462
Epoch 6/10
4/4 [==============================] - 0s 95ms/step - loss: 1.0095 - accuracy: 0.8155 - val_loss: 1.5898 - val_accuracy: 0.3462
Epoch 7/10
4/4 [==============================] - 0s 91ms/step - loss: 0.6107 - accuracy: 0.9320 - val_loss: 1.3565 - val_accuracy: 0.3462
Epoch 8/10
4/4 [====

# 2 conv layers  1 dense layer dropout (0.25, .24, 0.5 respectively)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_copy, labels_categorical, test_size=0.2, random_state=42)  # Split data

X_train = X_train.reshape(-1, 15028, 13, 1)
X_test = X_test.reshape(-1, 15028, 13, 1)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(15028, 13, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping])
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/100
4/4 [==============================] - 27s 6s/step - loss: 512.2645 - accuracy: 0.2524 - val_loss: 182.1291 - val_accuracy: 0.1923
Epoch 2/100
4/4 [==============================] - 23s 6s/step - loss: 360.5582 - accuracy: 0.2427 - val_loss: 21.5316 - val_accuracy: 0.2692
Epoch 3/100
4/4 [==============================] - 24s 6s/step - loss: 66.1820 - accuracy: 0.3010 - val_loss: 7.2220 - val_accuracy: 0.2308
Epoch 4/100
4/4 [==============================] - 24s 6s/step - loss: 11.6040 - accuracy: 0.3883 - val_loss: 10.5839 - val_accuracy: 0.1923
Epoch 5/100
4/4 [==============================] - 23s 6s/step - loss: 7.7659 - accuracy: 0.4272 - val_loss: 3.8667 - val_accuracy: 0.1923
Epoch 6/100
4/4 [==============================] - 24s 6s/step - loss: 2.8227 - accuracy: 0.4175 - val_loss: 1.5330 - val_accuracy: 0.3077
Epoch 7/100
4/4 [==============================] - 22s 6s/step - loss: 1.2180 - accuracy: 0.5340 - val_loss: 1.4174 - val_accuracy: 0.2692
Epoch 8/100
4/4 [

# 2 conv layers dense layer dropout with **L2  regularization**

In [ ]:
from tensorflow.keras import regularizers
X_train, X_test, y_train, y_test = train_test_split(data_copy, labels_categorical, test_size=0.2, random_state=42)  # Split data
X_train = X_train.reshape(-1, 15028, 13, 1)
X_test = X_test.reshape(-1, 15028, 13, 1)


model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu',
                           kernel_regularizer=regularizers.l2(0.01),
                           input_shape=(15028, 13, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
                           kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu',
                          kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping])


loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/100
4/4 [==============================] - 25s 6s/step - loss: 446.2744 - accuracy: 0.2039 - val_loss: 91.5264 - val_accuracy: 0.2692
Epoch 2/100
4/4 [==============================] - 22s 5s/step - loss: 144.0443 - accuracy: 0.3107 - val_loss: 10.5344 - val_accuracy: 0.3462
Epoch 3/100
4/4 [==============================] - 24s 6s/step - loss: 40.9192 - accuracy: 0.2330 - val_loss: 6.3664 - val_accuracy: 0.1923
Epoch 4/100
4/4 [==============================] - 21s 5s/step - loss: 13.6864 - accuracy: 0.3398 - val_loss: 5.1914 - val_accuracy: 0.3462
Epoch 5/100
4/4 [==============================] - 23s 5s/step - loss: 5.6241 - accuracy: 0.3981 - val_loss: 5.6342 - val_accuracy: 0.3077
Epoch 6/100
4/4 [==============================] - 21s 5s/step - loss: 5.6287 - accuracy: 0.4951 - val_loss: 5.5530 - val_accuracy: 0.3462
Epoch 7/100
4/4 [==============================] - 23s 5s/step - loss: 5.3044 - accuracy: 0.5534 - val_loss: 5.6677 - val_accuracy: 0.3462
Epoch 8/100
4/4 [==

In [ ]:
# def normalize_data(data, max_length):
#   for i in range(len(data)):
#         if data[i].shape[0] < max_length:
#               # Pad shorter sequences with zeros
#           padding = np.zeros((max_length - data[i].shape[0], data[i].shape[1]))
#           data[i] = np.vstack((data[i], padding))
#         elif data[i].shape[0] > max_length:
#             # Truncate longer sequences
#           data[i] = data[i][:max_length]

#           # Normalize MFCC features between -1 and 1
#         data_min_val = np.min(data[i])
#         data_max_val = np.max(data[i])
#         data_range = data_max_val - data_min_val
#         data[i] = -1 + 2 * ((data[i] - data_min_val) / data_range)

#   return data

array([[[-1.        ,  0.73380252,  0.73380252, ...,  0.73380252,
          0.73380252,  0.73380252],
        [-1.        ,  0.73380252,  0.73380252, ...,  0.73380252,
          0.73380252,  0.73380252],
        [-1.        ,  0.73380252,  0.73380252, ...,  0.73380252,
          0.73380252,  0.73380252],
        ...,
        [ 0.73380252,  0.73380252,  0.73380252, ...,  0.73380252,
          0.73380252,  0.73380252],
        [ 0.73380252,  0.73380252,  0.73380252, ...,  0.73380252,
          0.73380252,  0.73380252],
        [ 0.73380252,  0.73380252,  0.73380252, ...,  0.73380252,
          0.73380252,  0.73380252]],

       [[-1.        ,  0.64675808,  0.64675808, ...,  0.64675808,
          0.64675808,  0.64675808],
        [-1.        ,  0.64675808,  0.64675808, ...,  0.64675808,
          0.64675808,  0.64675808],
        [-1.        ,  0.64675808,  0.64675808, ...,  0.64675808,
          0.64675808,  0.64675808],
        ...,
        [ 0.64675808,  0.64675808,  0.64675808, ...,  

In [ ]:
data

array([[[-6.02393616e+02,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-6.02393616e+02,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-6.02393616e+02,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-6.42114136e+02,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-6.42114136e+02,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e